In [ ]:
#feed the dataset in model
import cv2
import numpy as np
import pyttsx3 # for speak purpose

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h+10, x:x+w+10]

    return cropped_face

# Initialize Webcam
user_id = input('ENTER USER-ID : ') # to differentiate between user
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
#         face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/User_' + str(user_id) + "_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        face1 = cv2.flip(face,1)
        cv2.putText(face1, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face1)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
pyttsx3.speak("Collecting Samples Complete")
print("Collecting Samples Complete")

In [ ]:
#train the model
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import pyttsx3
# Get the training data we previously made
data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    user_id=int(os.path.split(image_path)[-1].split("_")[1])
    Labels.append(user_id)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
#saving our created model in a file
model.save('./ayush4.h5')
pyttsx3.speak("Model trained sucessefully,let's go ahead")
print("success")

In [ ]:
#test the model
import cv2
import numpy as np
import pywhatkit #for msg sending 
import time
import imghdr
import os
import threading
import smtplib
from email.message import EmailMessage
import pyttsx3
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

user_id=0
name = ""

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, [] 
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi 

def send_txt_mail():
    

    msg = EmailMessage()
    pyttsx3.speak("enter mail id ")
    t=input("mail id-")
    msg['subject'] = "Trial purpose"
    msg['from'] = t
    msg['to'] = "receiver mail id put here"
    msg.set_content(" Location Alert..! We've detected your face in lucknow. Kindly cross verify with your img")
    
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login("sender mail id","mail id password put here")
        server.send_message(msg)
    pyttsx3.speak("text mail sent ,good job")    
    print("text mail sent ,good job")
    
def send_img_mail():
    From = "Face_Detector_Machine"
    Reciever_Email = "receiver mail id put here"

    msg = EmailMessage()                         
    msg['Subject'] = "Your face has detected..!" 
    msg['From'] = From
    msg['To'] = Reciever_Email 

    time.sleep(3)


    with open('pic.jpg', 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    msg.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)


    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login("sender mail id", "mail id password")
        server.send_message(msg)
        
    pyttsx3.speak("image mail also sent my sir ")
    print("image mail also sent")

def show(name,msg):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        image, face = face_detector(frame)
        image1 = cv2.flip(image,1)
        cv2.putText(image1, "Hey " +name+ " " +msg, (25, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image1 )
        if cv2.waitKey(1) == 13:
            break
    cap.release()
    cv2.destroyAllWindows()
# Open Webcam
cap = cv2.VideoCapture(0)
final = cv2.face_LBPHFaceRecognizer.create();
final.read('./ayush4.h5') #loading the  saved model   
    
while True:
    
    ret, frame = cap.read()
    #cv2.imwrite("pic.jpg", frame)
    image, face = face_detector(frame)  
    image = np.array(image)
    face = np.array(face)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = final.predict(face)
        user_id = results[0]
        score = results[1]

        if score < 500:
            confidence = int( 100 * (1 - (score)/400) )
            display_string = str(confidence) + '% Confident it is User'

        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

        if user_id == 1:
            name = "Ayush"
            msg="Check,your mail and inbox "
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            pyttsx3.speak("what u want to do sir")
            q=input()
            send_txt_mail()
            send_img_mail()
            t = time.localtime()
            time_hr = time.strftime("%H", t)
            time_min = time.strftime("%M", t)
            pywhatkit.sendwhatmsg("phone number", "Face_caputered in lucknow" , int(time_hr), int(time_min)+1,wait_time=10)
            break

        elif user_id == 2:
            name = "Shubhang"
            msg = "Hey ayush friend , Check AWS"
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            a = os.system("aws ec2 run-instances  --image-id ami-06a0b4e3b7eb7a320  --count 1  --instance-type t2.micro --key-name ayush11 --subnet-id subnet-a63fcbcd    --tag-specifications  ResourceType=instance,Tags=[{Key=AYUSH1,Value=DEVOPS1}]")
            if a == 0:
                pyttsx3.speak("\t\t\t Success : Instance Launched")
                print("\t\t\t Success : Instance Launched")
            b = os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1a  --tag-specifications  ResourceType=volume,Tags=[{Key=Name,Value=MLVOL1}]")
            if b == 0:
                pyttsx3.speak(" Success : Volume Created")
                print(" Success : Volume Created")
            import subprocess
            import time
            time.sleep(30)
            inst_id = subprocess.getoutput("aws ec2 describe-instances --filters Name=tag:AYUSH1,Values=DEVOPS1 --query Reservations[*].Instances[*].[InstanceId] --output text")
            volu_id = subprocess.getoutput("aws ec2 describe-volumes  --filters Name=tag:Name,Values=MLVOL1* --query Volumes[*].[VolumeId] --output text")
            c = os.system("aws ec2 attach-volume --instance-id {} --volume-id {} --device /dev/sdf".format(inst_id,volu_id))
            if c == 0:
                pyttsx3.speak(" Success : Volume ATTACHED")
                print("Success : Volume ATTACHED")
            break
            
        else:
            name = "Unknown"
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            break
            
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()